In [45]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [46]:
df = pd.read_csv("../input/riiid-test-answer-prediction/questions.csv")

In [47]:
df

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38
...,...,...,...,...,...
13518,13518,13518,3,5,14
13519,13519,13519,3,5,8
13520,13520,13520,2,5,73
13521,13521,13521,0,5,125


In [66]:
tag = df["tags"].str.split(" ", n=10, expand=True)
tag.columns = [f"tags{i}" for i in range(1, len(tag.columns) + 1)]
tags = np.arange(1, 188)
for col in tag.columns:
    tag[col] = pd.to_numeric(tag[col], errors='coerce').fillna(-1).astype("uint8")


In [67]:
target_cols = []
for t in tags:
    target_cols.append(f"tag_{t}")
    df[f"tag_{t}"] = (tag == t).sum(axis=1).astype("uint8")

# 生データをクラスタリング

In [68]:
from sklearn.cluster import KMeans

In [69]:
df["km20"] = KMeans(n_clusters=50).fit_predict(df[target_cols].values)

In [70]:
df[df["km20"] == 0].groupby(["tags", "part"]).size().sort_values().tail(100)

tags        part
125         6         1
123         6         2
61 110 29   1         2
44          6         2
75          6         2
95          6         2
127         6         2
43          6         2
132         6         2
156         6         3
174         6         3
152         6         3
151         6         3
55 162      6         4
115         6         4
134         6         4
48          6         4
49          6         4
37          7         4
54          6         4
133         6         4
72          6         4
80          6         4
180         6         5
33          6         5
166         6         5
34          5         6
61 110 102  1         6
78          5         6
177         6         6
181         6         7
4           6         7
25          6         7
64          6         7
45          6         8
47          6         8
172         5         9
58          5        10
179         5        10
23          5        10
112         5        10

In [71]:
df.groupby(["km20", "part"]).size()

km20  part
0     1          8
      5       1866
      6        284
      7          5
1     1          9
      2         13
      3        128
      4        181
2     1         24
      2         30
      3        110
      4         87
3     5        512
      6        118
4     2        289
5     2         37
      3        279
      4         17
6     5        348
      6         74
7     7        298
8     5        697
      6         54
9     5        417
      6        127
10    6        302
11    1        169
12    1          5
      2          7
      3          4
      4        154
13    1        187
14    2        135
      3         92
      4         21
15    3          4
      4        138
16    5        196
      6         30
17    3        149
      4        153
18    3         81
      4        143
19    7        124
20    2        197
21    2         96
22    5        330
      6         48
23    7        127
24    1        294
25    1          3
      4          3
 

8                    738
73                   617
53                   523
1                    413
96                   373
                    ... 
106 183 162 92         1
98 118 46 135          1
145 42 35 117 122      1
170 162                1
157 150 102            1
Name: tags, Length: 1519, dtype: int64

In [90]:
    df_question = pd.read_csv("../input/riiid-test-answer-prediction/questions.csv")
    tag = df_question["tags"].str.split(" ", n=10, expand=True)
    tag.columns = [f"tags{i}" for i in range(1, len(tag.columns) + 1)]
    tags = np.arange(1, 188)
    for col in tag.columns:
        tag[col] = pd.to_numeric(tag[col], errors='coerce').fillna(-1).astype("uint8")
    target_cols = []
    for t in tags:
        target_cols.append(f"tag_{t}")
        df_question[f"tag_{t}"] = (tag == t).sum(axis=1).astype("uint8")
    for p in [1, 2, 3, 4, 5, 6, 7]:
        target_cols.append(f"part_{p}")
        df_question[f"part_{p}"] = (df_question["part"] == p).astype("uint8") / 3
    df_question["kmeans_tags20"] = KMeans(n_clusters=10).fit_predict(df_question[target_cols].values)

In [93]:
df_question[df_question["kmeans_tags20"] == 7].groupby(["part", "tags"]).size()

part  tags 
5     8        697
6     8         41
      8 162     13
dtype: int64

In [92]:
df_question.groupby(["kmeans_tags20", "part"]).size().head(50)

kmeans_tags20  part
0              1        231
               2        117
               3          6
               4        423
1              5       4302
               6       1040
               7          7
2              2        224
               3        294
3              1        468
               2        253
               3        305
               4        575
4              1         43
               2        575
               3        248
               4         99
5              2        225
               3        306
6              7       1153
7              5        697
               6         54
8              5        512
               6        118
9              1        250
               2        253
               3        403
               4        342
dtype: int64